In [1]:
import pandas as pd
import pandas_profiling as pp
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv("../data/train.csv", )

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (8,11,12,13,14,15,16,17,18,19,21,23,26,28,29,30,32,82,83,85,91,269,325,326,327,328) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cat_columns = train.select_dtypes(exclude=[np.number]).columns
object_cols = {}
for col in cat_columns:
        object_cols[col] = 'str'

In [5]:
train = pd.read_csv("../data/train.csv", engine='c', sep=',', dtype=object_cols)
y = train["RESPONDERS"]
train.drop("RESPONDERS", axis=1, inplace=True)


test = pd.read_csv("../data/test.csv", engine='c', sep=',', dtype=object_cols)

In [49]:
## pp.ProfileReport(train[train.columns[:100]])

/usr/local/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}
/usr/local/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}
/usr/local/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}
/usr/local/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}
/usr/local/lib/python3.6/site-packages/pandas_profiling/base.py:223: RuntimeWarning: divide by zero encountered in long_scalars
  'p_unique': distinct_count / count}


Number of variables,100
Number of observations,300000
Total Missing (%),78.2%
Total size in memory,228.9 MiB
Average record size in memory,800.0 B
Numeric,40
Categorical,52
Date,0
Text (Unique),0
Rejected,8
Distinct count,2


In [6]:
numeric_cols = train.select_dtypes(include=[np.number]).columns
numeric_cols = [col for col in numeric_cols if col != 'CUSTOMER_ID']

In [11]:
cat_columns = train.select_dtypes(exclude=[np.number]).columns

In [13]:
%%time

from collections import defaultdict
d = defaultdict(LabelEncoder)

# fit and encode train/test
a = pd.concat([train[cat_columns], test[cat_columns]], axis=0).fillna('').apply(lambda x: d[x.name].fit(x))

# transform encodings to train
train[cat_columns] = train[cat_columns].fillna('').apply(lambda x: d[x.name].transform(x))

# transform encodings to test
test[cat_columns] = test[cat_columns].fillna('').apply(lambda x: d[x.name].transform(x))

CPU times: user 1min 23s, sys: 7.86 s, total: 1min 31s
Wall time: 1min 31s


In [14]:
y.value_counts(normalize=True)

N    0.984627
Y    0.015373
Name: RESPONDERS, dtype: float64

In [15]:
y = y.map({"Y": 1, "N": 0})

In [16]:
import lightgbm as lgb

params = {
    'application': 'binary',
    'num_leaves': 255,
    'feature_fraction': 0.6,
    'sub_row': 0.6,
    'bagging_freq': 1,
    'max_bin': 255,
    'metric': 'auc',
    'num_threads': 4,
    'verbose': 0,
    'seed': 42
}

num_rounds = 1000



In [17]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
validation = []
for train_indexes, val_indexes in skf.split(train, y):
    validation.append((train_indexes, val_indexes))

In [22]:
train_cols = numeric_cols + list(cat_columns)

In [23]:
lgb_train = lgb.Dataset(train[train_cols], y)
cv = lgb.cv(params, lgb_train, num_rounds, folds=validation, early_stopping_rounds=15, verbose_eval=1)

[1]	cv_agg's auc: 0.793883 + 0.0162781
[2]	cv_agg's auc: 0.818867 + 0.00528289
[3]	cv_agg's auc: 0.830422 + 0.00280652
[4]	cv_agg's auc: 0.853054 + 0.00437363
[5]	cv_agg's auc: 0.8534 + 0.00423033
[6]	cv_agg's auc: 0.852279 + 0.00324774
[7]	cv_agg's auc: 0.856493 + 0.00276448
[8]	cv_agg's auc: 0.857623 + 0.00283426
[9]	cv_agg's auc: 0.858376 + 0.00264556
[10]	cv_agg's auc: 0.863864 + 0.00222535
[11]	cv_agg's auc: 0.867988 + 0.00245649
[12]	cv_agg's auc: 0.867348 + 0.00216562
[13]	cv_agg's auc: 0.867733 + 0.00201587
[14]	cv_agg's auc: 0.868192 + 0.00204012
[15]	cv_agg's auc: 0.868894 + 0.00238181
[16]	cv_agg's auc: 0.869441 + 0.00234449
[17]	cv_agg's auc: 0.870112 + 0.00269449
[18]	cv_agg's auc: 0.871709 + 0.00297305
[19]	cv_agg's auc: 0.872215 + 0.00269287
[20]	cv_agg's auc: 0.871709 + 0.00276558
[21]	cv_agg's auc: 0.871111 + 0.00226233
[22]	cv_agg's auc: 0.87019 + 0.00223259
[23]	cv_agg's auc: 0.87432 + 0.00304669
[24]	cv_agg's auc: 0.874197 + 0.00329473
[25]	cv_agg's auc: 0.876599 + 

In [24]:
def custom_metric_lift(df):
    return df.sort_values('PROBABILITY', ascending=False).iloc[:int(len(temp_df)*0.1), 1].sum() / df["RESPONDERS"].sum()

In [33]:
scores = []
for train_indexes, val_indexes in validation:
    temp_df = pd.concat([train.loc[val_indexes,["CUSTOMER_ID"]],y.loc[val_indexes]], axis=1)
    model = lgb.train(params, lgb_train.subset(train_indexes), 94)
    temp_df["PROBABILITY"] = model.predict(train.loc[val_indexes, train_cols])
    scores.append(custom_metric_lift(temp_df))
    print("Score is {}".format(scores[-1]))

Score is 0.6695557963163596
Score is 0.6403033586132177
Score is 0.6529284164859002
Score is 0.6540130151843818
Score is 0.6822125813449024


In [34]:
np.mean(scores), np.std(scores)

(0.6598026335889523, 0.014548719587932423)

# Predict test

In [35]:
model = lgb.train(params, lgb_train, num_boost_round=91)

In [36]:
submission = pd.read_csv("../data/sample-submission.csv")

In [37]:
test["RESPONDERS"] = model.predict(test[train_cols])

In [38]:
test[submission.columns].to_csv("../predictions/baseline.csv", index=False)